In [1]:
import json
import numpy as np
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import os
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import requests

# Go get geckodriver from : https://github.com/mozilla/geckodriver/releases

In [14]:
def ffx_preferences(dfolder, download=False):
    '''
    Sets the preferences of the firefox browser: download path.
    '''
    profile = webdriver.FirefoxProfile()
    # set download folder:
    profile.set_preference("browser.download.dir", dfolder)
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf, application/csv,text/csv,image/png ,image/jpeg, application/pdf, text/html,text/plain,application/octet-stream")
    
    profile.add_extension('/Users/ruimaciel/Desktop/Barcelona/NLP_I/TA_sessions/2/uBlock0@raymondhill.net.xpi')

    # this allows to download pdfs automatically
    if download:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/x-pdf")
        profile.set_preference("pdfjs.disabled", True)

    options = Options()
    options.profile = profile
    return options


def start_up(link, dfolder, geko_path,donwload=True):
    geko_path='/Users/ruimaciel/Desktop/Barcelona/NLP_I/TA_sessions/2/geckodriver'
    download_path='/Users/ruimaciel/Downloads'
    os.makedirs(dfolder, exist_ok=True)

    options = ffx_preferences(dfolder,donwload)
    service = Service(geko_path)
    browser = webdriver.Firefox(service=service, options=options)
    # Enter the website address here
    browser.get(link)
    time.sleep(5)  # Adjust sleep time as needed
    return browser


def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(1)
        ss += 1
        if ss == 15:
            # warn_sound()
            # return NoSuchElementException
            ck = True
            # browser.quit()

def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element('xpath',xpath).click()
        elif type == "id":
            browser.find_element('id',xpath).click()
        elif type == "css":
            browser.find_element('css selector',xpath).click()
        elif type == "class":
            browser.find_element('class name',xpath).click()
        elif type == "link":
            browser.find_element('link text',xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(e)
        return False
    return True

In [20]:
# lets open booking:
dfolder='/Users/ruimaciel/Downloads'
geko_path='/Users/ruimaciel/Desktop/Barcelona/NLP_I/TA_sessions/2/geckodriver'
link='https://www.booking.com/index.en.html'

browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path)

#eliminate the cookie bar
browser.find_element(by='xpath', value='//*[@id="onetrust-reject-all-handler"]').click()

In [21]:
place = 'Barcelona'
search1 = browser.find_element(by='xpath',value='//*[@id=":re:"]')
search1.send_keys(place)

In [22]:
place = "Barcelona"
search1 = browser.find_element(by='xpath',value='//*[@id=":re:"]')
search1.send_keys(place)

from_day = '2024-05-25'
to_day = '2024-06-02'  # Change to your 'to_day'

calendar = 'button.ebbedaf8ac:nth-child(2) > span:nth-child(1)'

browser.find_element('css selector', calendar).click()

path = '//div[@id="calendar-searchboxdatepicker"]//table[@class="eb03f3f27f"]//tbody//td[@class="b80d5adb18"]//span[@class="cf06f772fa"]'
found_from_day = False
found_to_day = False
is_first_iteration = True
next_button_path_1 = '.f4552b6561'
next_button_path_2 = 'button.f38b6daa18:nth-child(2)'

while not (found_from_day and found_to_day):
    dates = browser.find_elements('xpath', path)
    for date in dates:
        if date.get_attribute("data-date") == from_day and not found_from_day:
            date.click()
            found_from_day = True
            break

        if date.get_attribute("data-date") == to_day and not found_to_day:
            date.click()
            found_to_day = True
            break

    if not (found_from_day and found_to_day):
        if is_first_iteration:
            browser.find_element('css selector', next_button_path_1).click()
            is_first_iteration = False
        else:
            browser.find_element('css selector', next_button_path_2).click()


search_button='/html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span'

check_obscures(browser,search_button , type='xpath')
check_and_click(browser,search_button , type='xpath')


Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/E

In [23]:
def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath',
        "/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[4]/div[2]/nav/nav/div/div[2]/ol/li[7]/button")
    return(int(a[-1].text))

pages = get_number_pages(browser)

print(pages)

40


In [30]:
pages=1

change_page_xpath = '/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[4]/div[2]/nav/nav/div/div[3]/button/span/span'
css = 'div.b16a89683f:nth-child(3) > button:nth-child(1) > span:nth-child(1) > span:nth-child(1)'
data = pd.DataFrame(columns=['Hotels', 'Ratings', 'Descriptions', 'Price', 'Link'])

# Go to first page
first_page_xpath='/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[4]/div[2]/nav/nav/div/div[2]/ol/li[1]/button'
check_and_click(browser,first_page_xpath , type='xpath')
for i in range(pages):
    # Save hotels info
    print(f'Page: {i + 1}')
    containers = browser.find_elements('xpath', '//div[@class="c066246e13"]')
    for hotel in containers:
        hotel_name = hotel.find_element('xpath', './/div[@class="f6431b446c a15b38c233"]').text
        try:
            hotel_rating = hotel.find_element('xpath', './/div[@class="a3b8729ab1 d86cee9b25"]').text
        except:
            hotel_rating = np.nan
        try:
            promotion = hotel.find_element('xpath', './/div[@class="d17181842f"]').text
        except: 
            promotion = np.nan
        try:
            hotel_description_short= hotel.find_element('xpath', './/<div[@class="abf093bdfe e8f7c070a7"]').text
        except:
            hotel_description_short = np.nan
        try:
            hotel_description_long = hotel.find_element('xpath', './/div[@class="abf093bdfe d323a31618"]').text
        except:
            hotel_description_long = np.nan
        try:
            hotel_price = hotel.find_element('xpath', './/span[@class="f6431b446c fbfd7c1165 e84eb96b1f"]').text
        except:
            hotel_price = np.nan
        try:
            url = hotel.find_element('xpath', './/a[@href]')
            hotel_url= url.get_attribute('href')
        except:
            hotel_url = np.nan
    
        new_row = {'Hotels': hotel_name, 'Ratings': hotel_rating, 'Promotion': promotion, 'Short Description': hotel_description_short, 'Long Descriptions': hotel_description_long, 'Price':hotel_price, 'Link': hotel_url}
        data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)

    # Change page with CSS Selector
    browser.find_element('css selector', css).click()
    time.sleep(2)
display(data)

<div class="c59cd18527"><h4 role="link" tabindex="0" class="abf093bdfe e8f7c070a7">One-Bedroom Apartment</h4><ul class="ba51609c35"><li class="a6a38de85e"><div class="fc367255e6"><span class="f419a93f12"></span><div class="abf093bdfe">1 double bed</div></div></li><li class="a6a38de85e"><div class="aca0ade214 aaf30230d9 c2931f4182 cd0091cad1 bca29be503"><div class=""><span data-testid="cancellation-policy-icon" class="fcd9eec8fb e926b2a7d7 bf9a32efa5 a229b4a525" aria-hidden="true"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 128 128"><path d="M56.33 102a6 6 0 0 1-4.24-1.75L19.27 67.54A6.014 6.014 0 1 1 27.74 59l27.94 27.88 44-58.49a6 6 0 1 1 9.58 7.22l-48.17 64a5.998 5.998 0 0 1-4.34 2.39z"></path></svg></span></div><div class="e7a57abb1e afd4cc3363"><div class="abf093bdfe d068504c75"><strong>Free cancellation</strong></div></div></div></li><li class="a6a38de85e"><div><div class="d17181842f">Only 1 left at this price on our site</div></div></li></ul></div>

Page: 1


,Hotels,Ratings,Descriptions,Price,Link,Promotion,Short Description,Long Descriptions
0,Golden Hotel Barcelona,NaN,NaN,"€ 1,399",https://www.booking.com/hotel/es/one-bedroom-a...,Only 1 left at this price on our site,NaN,NaN
1,Sonder Casa Luz,8.4,NaN,"€ 2,540",https://www.booking.com/hotel/es/casa-luz-barc...,Only 1 room left at this price on our site,NaN,NaN
2,Hotel Viladomat by Silken,8.1,NaN,"€ 2,049",https://www.booking.com/hotel/es/pereiv.en-gb....,NaN,NaN,NaN
3,Hotel del Mar,8.3,NaN,"€ 1,987",https://www.booking.com/hotel/es/hotelalimara....,Only 1 left at this price on our site,NaN,Private suite
4,Four Points by Sheraton Barcelona Diagonal,8.3,NaN,"€ 2,099",https://www.booking.com/hotel/es/aparthotelnap...,NaN,NaN,Entire apartment • 1 bedroom • 1 bathroom • 1 ...
5,Hotel Derby,8.0,NaN,"€ 1,812",https://www.booking.com/hotel/es/lloret.en-gb....,Only 4 rooms left at this price on our site,NaN,NaN
6,Barcelona Centric,8.2,NaN,"€ 2,111",https://www.booking.com/hotel/es/concordia.en-...,NaN,NaN,NaN
7,Hotel Lloret Ramblas,8.2,NaN,"€ 1,797",https://www.booking.com/hotel/es/barnapartment...,Only 1 left at this price on our site,NaN,Entire apartment • 1 bedroom • 1 living room •...
8,AC Hotel Irla by Marriott,9.3,NaN,"€ 2,775",https://www.booking.com/hotel/es/the-streets-a...,Only 5 left at this price on our site,NaN,Entire apartment • 1 bedroom • 1 living room •...
9,The Streets Apartments Rambla,7.5,NaN,"€ 2,466",https://www.booking.com/hotel/es/you-stylish-l...,Only 1 left at this price on our site,NaN,Entire apartment • 1 bedroom • 1 living room •...


In [25]:
data

,Hotels,Ratings,Descriptions,Price,Link,Promotion
0,El Avenida Palace,8.5,NaN,"€ 2,853",https://www.booking.com/hotel/es/catalunyaplaz...,Only 5 rooms left at this price on our site
1,Stay U-nique Apartments Balmes,NaN,NaN,"€ 6,393",https://www.booking.com/hotel/es/canela-homes-...,NaN
2,Habitat Apartments Barcelona Plaza,9.5,Entire apartment • 3 bedrooms • 1 living room ...,"€ 9,376",https://www.booking.com/hotel/es/sweet-inn-pla...,Only 1 left at this price on our site
3,Izaila Plaza Catalunya,8.8,NaN,"€ 2,669",https://www.booking.com/hotel/es/iberostar-pas...,NaN
4,Iberostar Selection Paseo de Gracia 4 Sup,8.2,Entire apartment • 2 bedrooms • 1 living room ...,"€ 3,963",https://www.booking.com/hotel/es/paseo-de-grac...,Only 1 left at this price on our site
...,...,...,...,...,...,...
445,Born Rooms - St Caterina,NaN,Entire apartment • 2 bedrooms • 1 living room ...,"€ 4,274",https://www.booking.com/hotel/es/sweet-inn-eth...,Only 1 left at this price on our site
446,CENTRAL STYLISH EIXAMPLE - Casa Antonio By MyR...,5.8,Entire apartment • 3 bedrooms • 1 living room ...,"€ 3,925",https://www.booking.com/hotel/es/habitat-apart...,Only 1 left at this price on our site
447,Habitat Apartments Batlló,9.7,Entire apartment • 3 bedrooms • 3 bathrooms • ...,"€ 7,525",https://www.booking.com/hotel/es/ola-living-pa...,Only 1 left at this price on our site
448,Stay U-nique Apartments Roger Llúria,7.9,Entire apartment • 2 bedrooms • 1 living room ...,"€ 5,649",https://www.booking.com/hotel/es/charming-apar...,Only 1 left at this price on our site
